<a href="https://colab.research.google.com/github/turtlenoise/simple_sentiment_analysis/blob/master/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import thinc.extra.datasets

imdb_data = thinc.extra.datasets.imdb()

sentences, labels = zip(*imdb_data[1])

83738624/84125825 [============================>.] - ETA: 0s Untaring file...


In [2]:
print(str(labels[0]) + " " + sentences[0])

0 It is truly saddening to see a once-great director such as Deodato delivering such a second-rate giallo such as this. This movie was so terrible it effectively put an end to his movie career. The box lies, this is no "erotic thriller", hell during the film's 97 minute running time, Charlotte Lewis barely shows us one nipple! I thought it would pick up once William Burger showed up (in one of his last roles), but his character is killed off rather abruptly and lamely. This movie fails in pretty much every way. Claudio Simonetti's music is little more than noise, and the plot made very little sense at all. For some reason, Lewis is terrorized by ghosts which attack using phones. (?) By the end, the characters all seem to have forgotten the previous 90 minutes of hell they went through, and casually laugh as they sick the evil spirits on someone else, Lewis's ex-boyfriend. What?!? This movie did little for me besides anger me.... and bore me half to death. For genuine 80's Deodato fun, 

In [3]:
negative_text = ""
for i in range(1,1500):
  if (labels[i] == 0):
    negative_text = negative_text + sentences[i]

In [340]:
import spacy
import re
nlp = spacy.load('en_core_web_sm')
negative_text = re.sub(r'[^\w\s]','',negative_text)  
corpus = nlp(negative_text)

In [342]:
def has_negative_children(token):
  for child in token.children:
    if (child.dep_ == 'neg'):
      return True
  return False 

# negative_adverb_list = []
# negative_adjective_list = []
negative_word_frequency = dict()
for sent in corpus.sents:
    for token in sent:
        if (token.is_stop == False):
          if (token.lemma_ in negative_word_frequency.keys()) and (has_negative_children(token) == False):
              negative_word_frequency[token.lemma_] += 1
          else:
              negative_word_frequency[token.lemma_] = 1
        # if (token.pos_ == 'ADJ') and (has_negative_children(token) == False):
        #   negative_adjective_list.append(token.lemma_)
        # if (token.pos_ == "ADV") and (has_negative_children(token) == False):
        #   negative_adverb_list.append(token.lemma_)

In [130]:
# cutRareWords = sorted(negative_word_frequency.items(), key=lambda item: item[1])[7000:-100]
# negative_word_frequency = dict(cutRareWords)

In [347]:
negative_word_frequency['bad']

500

In [346]:
negative_word_frequency['bad'] = 500

In [301]:
positive_text = ""
for i in range(1,1500):
  if (labels[i] == 1):
    positive_text = positive_text + sentences[i]

In [304]:
positive_text = re.sub(r'[^\w\s]','',positive_text)  
corpus = nlp(positive_text)

In [228]:
# positive_adverb_list = []
# positive_adjective_list = []
positive_word_frequency = dict()
for sent in corpus.sents:
    for token in sent:
        if (token.is_stop == False):
          if (token.lemma_ in positive_word_frequency.keys()) and (has_negative_children(token) == False):
              positive_word_frequency[token.lemma_] += 1
          else:
              positive_word_frequency[token.lemma_] = 1
        # if (token.pos_ == 'ADJ') and (has_negative_children(token) == False):
        #   positive_adjective_list.append(token.lemma_)
        # if (token.pos_ == 'ADV') and (has_negative_children(token) == False):
        #   positive_adverb_list.append(token.lemma_)

In [136]:
# cutRareWords = sorted(positive_word_frequency.items(), key=lambda item: item[1])[2000:-20]
# positive_word_frequency = dict(cutRareWords)

In [348]:
positive_word_frequency['good']

430

In [349]:
positive_word_frequency['bad']

5

In [230]:
positive_word_frequency['a']

KeyError: ignored

In [350]:
word_values = dict()
for word in negative_word_frequency:
  if (word not in word_values):
    word_values[word] = negative_word_frequency[word]

for word in positive_word_frequency:
  if (word in word_values):
    word_values[word] = positive_word_frequency[word] / (positive_word_frequency[word] + negative_word_frequency[word])
  else: 
    word_values[word] = 1

for word in negative_word_frequency:
    if (word_values[word] > 1):
      word_values[word] = 1


In [351]:
word_values['good']

0.9207708779443254

In [352]:
word_values['lousy']

0.375

In [353]:
word_values['blue']

0.5238095238095238

In [449]:
import re

def get_score(example_sentence):
  example_sentence = re.sub(r'[^\w\s]','',example_sentence)  
  corpus = nlp(example_sentence)
  sum = 0
  count = 0
  for sent in corpus.sents:
      for token in sent:

        if (token.lemma_ in word_values) and (token.pos_ == 'ADJ' or token.pos_ == 'ADV'):
          if (word_values[token.lemma_]) < 0.5:
            value = 0.11
          else: 
            value = 0.68
          # else: 
          #   value = 1
          sum = sum + 10*value
          count = count + 10
        # if (token.lemma_ in word_values):
        #   if (word_values[token.lemma_]) < 0.5:
        #     value = 0
        #   else:
        #     value = 1
        #   sum = sum + value
        #   count = count + 1
  
  if (count == 0):
      average_sentiment = 0.5
  else:
    average_sentiment = sum/count
  
  return average_sentiment


In [372]:
example_sentence = "Best movie I have ever seen, amazing job, the director is a genius."
get_score(example_sentence)

0.8806752940446266

In [373]:
example_sentence = "Scary, boring, totally disgusting, yucks."
get_score(example_sentence)

0.3762939569391182

In [374]:
example_sentence = "Best movie I have ever seen although it had some scary, boring and disgusting scenes."
get_score(example_sentence)

0.5470168660746819

In [450]:
correctPositive = 0
allPositive = 0
correctNegative = 0
allNegative = 0
for i in range(0,1000):
  value = get_score(sentences[i])  
  
  if (labels[i] == 0):
    allNegative += 1
    if (value < 0.5):
      correctNegative += 1
  if (labels[i] == 1):
    allPositive += 1
    if (value > 0.5):
      correctPositive += 1

accuracyPositive = correctPositive / allPositive
accuracyPositive * 100

74.79838709677419

In [451]:
accuracyPositive = correctPositive / allPositive
accuracyPositive * 100

74.79838709677419

In [452]:
accuracyNegative = correctNegative / allNegative
accuracyNegative * 100

81.94444444444444

In [453]:
accuracy = (accuracyNegative + accuracyPositive) / 2
accuracy*100

78.3714157706093

In [261]:

example_sentence = "the and a."
print(example_sentence)
get_score(example_sentence)

the and a.


0.5

In [262]:
word_values['a.']

2

In [382]:
word_values['']

0.4594594594594595